In [7]:
import pandas as pd

from config import DATA_FILE_PATH, CHUNKSIZE


data_chunks = pd.read_csv(DATA_FILE_PATH, chunksize= CHUNKSIZE)

In [21]:
# Load in chunks
num_chunks = 0

# Process chunks
for idx, chunk in enumerate(data_chunks): 
    num_chunks += 1

print(num_chunks)
print(idx)
print(chunk['canonical_smiles'])

0
74
37888    CC(=O)O[C@@]12CO[C@@H]1C[C@H](O)[C@@]1(C)C(=O)...
37889    CO[C@@]12[C@H](COC(N)=O)C3=C(C(=O)C(C)=C(N)C3=...
37890    CC[C@@]1(O)C(=O)OCc2c1cc1n(c2=O)Cc2cc3ccccc3nc2-1
37891    C[C@]12CC[C@H](O)C[C@@]1(O)CC[C@@H]1[C@@H]2CC[...
37892    O=C1c2ccccc2-c2nc3[nH]c(=O)[nH]c(=O)c3c(-c3ccc...
                               ...                        
38365    Cc1cccc(C)c1OCC(=O)NC[C@](O)(Cc1ccccc1)C(=O)N1...
38366    Cc1cccc(C)c1OCC(=O)NC[C@@](O)(Cc1ccccc1)C(=O)N...
38367    Cc1cccc(C)c1OCC(=O)NC[C@@](O)(Cc1ccccc1)C(=O)N...
38368    Cc1cccc(C)c1OCC(=O)NC[C@](O)(Cc1ccccc1)C(=O)N1...
38369    Cc1cccc(C)c1OCC(=O)NC[C@](O)(CCc1ccccc1)C(=O)N...
Name: canonical_smiles, Length: 482, dtype: object


In [16]:
class DataHandler: 
    def find_input_type(self, file_path=DATA_FILE_PATH):
        self.file_path = file_path
        if file_path.split('.')[-1] == 'csv':
            return True
        elif file_path.split('.')[-1] == 'txt':
            return False 
        else: 
            raise ValueError('Unsupported input file. Only .txt and .csv are supported')

In [22]:
data_handler = DataHandler()

data_handler.find_input_type(file_path='test.csv')



True

In [26]:
from tqdm import tqdm
import pickle


# Load all fingerprints with tqdm progress bar
fingerprints, smiles_list, features = [], [], []
for idx in tqdm(range(75), desc="Loading Fingerprints"):
    with open(f'data/fingerprints_chunk_{idx}.pkl', 'rb') as f:
        fps_chunk, smiles_chunk, features_chunk = pickle.load(f)
        fingerprints.extend(fps_chunk)
        smiles_list.extend(smiles_chunk)
        features.extend(features_chunk)

Loading Fingerprints: 100%|██████████| 75/75 [00:00<00:00, 177.31it/s]


In [33]:
from multiprocessing import Pool
from config import N_JOBS
from rdkit import Chem
from mhfp.encoder import MHFPEncoder
smiles = 'CCNC(=O)Nc1cn2c(-c3ncc(C)cn3)cc(-c3cncnc3)cc2n1'

encoder = MHFPEncoder(512)

def calculate_fingerprint(smiles): 
    """Calculate fp for a single SMILES string"""
    fingerprint = encoder.encode(smiles)
    print(fingerprint)
    print(len(fingerprint), type(fingerprint))
    

calculate_fingerprint(smiles)

[262082127  33988699 145399841   7536380  29533652  17833582  98973281
  36401732  59424947  30182378  10436506  66619231  24029013  11104027
  52209339 111089521 174338065  13211080  83435364  13101872  55912304
  38318159 106862503  17605790  64871883  36880799  52675566  98309013
  58215480   8910025  16544013  66810485  74025885   7979412  74101553
  15588211 121626273  20134412  51988843  62628366 118528504  27819614
  43832848  42713914  41225061   6474523  55862497 133026785  72126913
  25304192  17938670  70584151  60585663  89610833  47218640  73365134
 105183453  20615859  52567477 106306564  13701932  16381178 133642774
  28578668   1332291  13955396  14658249  48133865   1853046 103001043
  44506480  36982711   2611622  73429708  72754427  13657548 131131656
  33508515  63268730  77450973  18987137  36787483  75938248  85961606
 165650711  10311684  57372615  13152365  55330600 183890132   9525305
  12691500 107778755  51643882   5597317   3218179  10665798  20219046
   379